In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Importing Packages

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import tensorflow as tf
from tensorflow import keras
from keras.engine.sequential import Sequential
from keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D, AveragePooling2D
from keras.models import Sequential, load_model
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16
from keras import Model
from keras import metrics
import os

## Reading in Dataset to Flow

In [3]:
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator
path = "/content/drive/MyDrive"
batch_size = 32
image_length = 128
image_size = (image_length, image_length)

train_datagen = ImageDataGenerator(rescale = 1./255, validation_split = 0.3)

train_batches = train_datagen.flow_from_directory(
    path,
    target_size = image_size,
    batch_size = batch_size,
    class_mode = "categorical",
    color_mode = "grayscale",
    shuffle = True,
    seed = 4243,
    subset = "training"
)

validation_batches = train_datagen.flow_from_directory(
    path,
    target_size = image_size,
    batch_size = batch_size,
    class_mode = "categorical",
    color_mode = "grayscale",
    shuffle = True,
    seed = 4243,
    subset = "validation"
)

Found 3461 images belonging to 3 classes.
Found 1482 images belonging to 3 classes.


In [4]:
# For Visualization of images and labels
def plot_images(image_array):
    num_images_per_row = 5
    num_images = len(image_array)
    num_rows = num_images // num_images_per_row
    if num_rows * 5 < num_images: num_rows += 1
    fig, axes = plt.subplots(num_rows, num_images_per_row)
    axes = axes.flatten()
    for img, ax in zip(image_array, axes):
        ax.imshow(img)
        ax.axis('off')
    plt.tight_layout()
    plt.show()

LABELS = {0: "Carrying", 1: "Normal", 2: "Threat"}
def print_label(labels):
    if labels.shape == (3,):
        print(LABELS[np.where(labels == 1)[0][0]])
        return
    for label in labels:
        print(LABELS[np.where(label == 1)[0][0]])


In [ ]:
images, labels = train_batches[0]

In [ ]:
# plot_images(images)
# print_label(labels)

## Building Model

In [10]:
# Create a VGG 16 model
def create_model(type = "VGG16"):
    model = Sequential()

    model.add(Conv2D(input_shape=(image_length,image_length,1),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
    model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
    model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))

    model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))

    model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))

    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))

    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
    model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))

    model.add(Flatten())
    model.add(Dense(units=4096,activation="relu"))
    model.add(Dense(units=4096,activation="relu"))
    model.add(Dense(units=3, activation="softmax")) 

    return model

In [ ]:
# Cell to start training (DO NOT RUN UNLESS FIRST TIME TRAINING)
model = create_model()

# https://stackoverflow.com/questions/71799046/model-fit-gives-invalidargumenterror-graph-execution-error
model.compile(loss = 'categorical_crossentropy',
              optimizer = 'adam',
              metrics = ['accuracy', metrics.categorical_accuracy])

## Training Model

In [8]:
# https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html
# https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator

history = model.fit(
        train_batches,
        steps_per_epoch = train_batches.__len__(),
        epochs = 5,
        validation_data = validation_batches,
        validation_steps = validation_batches.__len__())


Epoch 1/5
109/109 [==============================] - 3659s 33s/step - loss: 1.1098 - accuracy: 0.3721 - categorical_accuracy: 0.3721 - val_loss: 1.0953 - val_accuracy: 0.3758 - val_categorical_accuracy: 0.3758
Epoch 2/5
109/109 [==============================] - 2865s 26s/step - loss: 1.0956 - accuracy: 0.3756 - categorical_accuracy: 0.3756 - val_loss: 1.0950 - val_accuracy: 0.3758 - val_categorical_accuracy: 0.3758
Epoch 3/5
109/109 [==============================] - 2890s 27s/step - loss: 1.0955 - accuracy: 0.3756 - categorical_accuracy: 0.3756 - val_loss: 1.0946 - val_accuracy: 0.3758 - val_categorical_accuracy: 0.3758
Epoch 4/5
109/109 [==============================] - 2901s 27s/step - loss: 1.0955 - accuracy: 0.3756 - categorical_accuracy: 0.3756 - val_loss: 1.0951 - val_accuracy: 0.3758 - val_categorical_accuracy: 0.3758
Epoch 5/5
109/109 [==============================] - 2891s 27s/step - loss: 1.0953 - accuracy: 0.3756 - categorical_accuracy: 0.3756 - val_loss: 1.0948 - val_ac

From the training accuracy above, we see that training accuracy converges after 2 epochs. Thus, we can infer that the low accuracy has nothing to do with the number of epochs (as we have already reached convergence). It could be due to:
1) Low model complexity (VGG16 is a shallow model with only 2 layers)
2) Small image size (resizing to 128x128 removes important information)

It is likely not due to lack of image augmentation techniques, which are only added to allow the model to better generalize to testing images.

## Saving Model (Not working on Colab)

In [9]:
# Not working on Colab
model.save_weights('./checkpoints/my_checkpoint')

In [ ]:
# Cell to continue training
# Not working on Colab

model = create_model()
model.load_weights('./checkpoints/my_checkpoint')

model.compile(loss = 'categorical_crossentropy',
              optimizer = 'adam',
              metrics = ['accuracy', metrics.categorical_accuracy])

## Misc

In [ ]:
# Transfer Learning (To Read Later. Can use ImageNet pre-trained model to (possibly) speed up convergence)
# VGG 16: https://keras.io/api/applications/vgg/
# https://github.com/bnsreenu/python_for_microscopists/blob/master/Tips_tricks_20_Understanding%20transfer%20learning%20for%20different%20size%20and%20channel%20inputs.py